In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
#from statsmodels.stats.weightstats import DescrStatsW
from matplotlib import patheffects
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Date, MetaData, Table, Numeric, Time
import sys
from urllib.request import urlopen
import json
import Funciones

host = "localhost"
database = "dane"
user = "postgres"
port = 5432
password = "1234"

data_path = r"D:\Despliegue\Proyecto HOP\Data\Vcalle/"
dicc_path = r"D:\Despliegue\Proyecto HOP\Data/"

connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
connection_string


## Conectándose con el motor e importando las tabñas
engine = create_engine(connection_string)
insp = inspect(engine)
insp.get_table_names()


sql = 'SET SEARCH_PATH TO public, data_cruda, data_intermedia; SELECT * FROM v_calle;'

df = pd.DataFrame(pd.read_sql(sql, con = engine))

In [3]:
df.head()

,uid_encuesta,cod_dpto,cod_mpio,clase,meses_trabajo,semanas_trabajo,dias_trabajo,horastra,lugar_fijo,estaciona,...,cual_prog,puestos_2023,ididenti,idnit,iddv,obs_ix,obs_x,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,31572,81,81065,2,14. Diciembre,1. Una semana,5. Viernes,5,2. No,1. SÃ­,...,NaN,2.0,2.0,8.0,11.0,NaN,1.000000e+09,10.0,2.135600e+12,8.542778e+10
1,17809,18,18256,1,11. Septiembre,1. Una semana,6. SÃ¡bado,6,2. No,2. No,...,NaN,1.0,7.0,12.0,5.0,NaN,1.000000e+13,NaN,1.800450e+12,NaN
2,25458,18,18753,2,7. Mayo,3. Tres semanas,6. SÃ¡bado,6,2. No,1. SÃ­,...,NaN,1.0,9.0,NaN,7.0,NaN,1.000000e+08,NaN,2.497190e+11,NaN
3,9162,18,18256,1,11. Septiembre,2. Dos semanas,5. Viernes,6,2. No,1. SÃ­,...,NaN,2.0,3.0,7.0,9.0,NaN,NaN,NaN,NaN,NaN
4,43946,15,15176,2,13. Noviembre,1. Una semana,3. MiÃ©rcoles,11,1. SÃ­,2. No,...,NaN,3.0,8.0,12.0,7.0,NaN,NaN,NaN,NaN,NaN


In [4]:
rel_g12 = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="GRUPOS12")
rel_sex = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="SEXO")
rel_anio = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="ANOS_ESPACIO")

dic_depto = {5:"Antioquia",8:"Atlántico",11:"Bogotá, D. C.",13:"Bolívar",15:"Boyacá",17:"Caldas",18:"Caquetá",
             19:"Cauca",20:"Cesar",23:"Córdoba",25:"Cundinamarca",27:"Chocó",41:"Huila",44:"La Guajira",47:"Magdalena",
             50:"Meta",52:"Nariño",54:"Norte De Santander",63:"Quindío",66:"Risaralda",68:"Santander",70:"Sucre",73:"Tolima",
             76:"Valle Del Cauca",88:"San Andrés"}

dic_sector = {2:"Industria",3:"Comercio",4:"Servicios"}

dic_rut = {1:"Sí",0:"No"}

dic_g12 = {3:"Industria",
           4:"Construcción",
           5:"Comercio",
           6:"Transporte y almacenamiento",
           7:"Alojamiento/servicios de comida",
           8:"Información y comunicaciones",
           9:"Actividades inmobiliarias",
           10:"Educación",
           11:"Actividades de salud humana",
           12:"Actividades artísticas",
}

dic_ANOS_ESPACIO = {1: "[0-1)", 
                    2: "[1-3)",
                    3: "[3-5)",
                    4: "[5-10)",
                    5: "x>=10"} 

var_mov = ['uid_encuesta','ventas','pottot','cyg','va','c_v','cod_dpto','cpam','grupos12','anos_espacio','hombre'] 

'''
Creación de variables
'''

# Crear la variable de Costos y Gastos agregada 
df['cyg'] = df[['gasto_tot','costos']].sum(axis=1)
df.loc[(df['gasto_tot'].isna()) & (df['costos'].isna()), 'cyg'] = np.nan

# Total Personal Ocupado
df['trabajsoc'] = pd.to_numeric(df['trabajsoc'], errors= 'coerce')
df['trabajpago'] = pd.to_numeric(df['trabajpago'], errors= 'coerce')
df['trabajfami'] = pd.to_numeric(df['trabajfami'], errors= 'coerce')

df['pottot'] = df[['trabajsoc','trabajpago','trabajfami']].sum(axis=1)
df.loc[(df['trabajsoc'].isna()) & (df['trabajpago'].isna()) & (df['trabajfami'].isna()), 'cyg'] = np.nan

# Ciudad Principal y Área Metropolitana
cpam = [11001,	5001,	5079,	5088,	5129,	5212,	5266,	5308,	5360,	5380,	5631,	76001,	76892,	8001,
        8758,	68001,	68276,	68307,	68547,	17001,	17873,	52001,	66001,	66170,	66400,	73001,	54001,	54874,
        54405,	54261,	50001,	23001,	13001,	15001,	18001,	19001,	20001,	27001,	41001,	44001,	47001,	63001,	
        70001,	88001]
df['cod_mpio'] = pd.to_numeric(df['cod_mpio'], errors= 'coerce')
df['cpam'] = (df['cod_mpio'].isin(cpam) & df['clase']==1).apply(lambda x: int(x))

# Actividad Económica
df['division'] = df['ciiu_4'].apply(Funciones.extract_first_two_digits)
df = pd.merge(df,rel_g12,on='division',how="left")

# Valor Agregado
df['va'] = df['ventas'] - df['cyg']

# Coeficiente técnico de costos y gastos sobre ingresos
df['c_v'] = df['cyg'] / (df['ventas'] + 1)

'''
Modificación de los labels de algunas variables
'''
# Antigüedad
df['anos_espacio'] = df['anos_espacio'].map(dict(zip(rel_anio['ANOS_ESPACIO'], rel_anio['Codigo'])))

# Sexo
df['sexo'] = df['sexo'].map(dict(zip(rel_sex['SEXO'], rel_sex['Codigo'])))
df.rename(columns = {'sexo' : 'hombre'}, inplace = True)


'''
Selección de las variables
'''
df_final = df[var_mov]


'''
Filtrado de las unidades económicas
'''
# Ubicación geográfica: departamentos
depto_emicron = [8, 13, 44, 47, 70, 20, 25, 11, 18, 41, 15, 68, 54, 76, 52, 19, 17, 63, 73, 66,  5, 23, 27, 50, 88]
df_final = df_final[df_final['cod_dpto'].isin(depto_emicron)]

# Descartar unidades económicas con algún NA o 99 dentro de sus registros
id_ig = df_final['uid_encuesta'][((df_final.isna()) | (df_final==99)).any(axis=1)]
df_final = df_final[~(df_final['uid_encuesta'].isin(id_ig))]

'''
Asignar labels
'''
df_final['cod_dpto'] = df_final['cod_dpto'].map(dic_depto)
df_final['cpam'] = df_final['cpam'].map(dic_rut)
df_final['grupos12'] = df_final['grupos12'].map(dic_g12)
df_final['hombre'] = df_final['hombre'].map(dic_rut)
df_final['anos_espacio'] = df_final['anos_espacio'].map(dic_ANOS_ESPACIO)

In [11]:
rel_g12 = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="GRUPOS12")
rel_sex = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="SEXO")
rel_anio = pd.read_excel(dicc_path + "Diccionario.xlsx", sheet_name="ANOS_ESPACIO")

dic_depto = {5:"Antioquia",8:"Atlántico",11:"Bogotá, D. C.",13:"Bolívar",15:"Boyacá",17:"Caldas",18:"Caquetá",
             19:"Cauca",20:"Cesar",23:"Córdoba",25:"Cundinamarca",27:"Chocó",41:"Huila",44:"La Guajira",47:"Magdalena",
             50:"Meta",52:"Nariño",54:"Norte De Santander",63:"Quindío",66:"Risaralda",68:"Santander",70:"Sucre",73:"Tolima",
             76:"Valle Del Cauca",88:"San Andrés"}

dic_sector = {2:"Industria",3:"Comercio",4:"Servicios"}

dic_rut = {1:"Sí",0:"No"}

dic_g12 = {3:"Industria",
           4:"Construcción",
           5:"Comercio",
           6:"Transporte y almacenamiento",
           7:"Alojamiento/servicios de comida",
           8:"Información y comunicaciones",
           9:"Actividades inmobiliarias",
           10:"Educación",
           11:"Actividades de salud humana",
           12:"Actividades artísticas",
}

dic_ANOS_ESPACIO = {1: "[0-1)", 
                    2: "[1-3)",
                    3: "[3-5)",
                    4: "[5-10)",
                    5: "x>=10"} 

var_mov = ['uid_encuesta','ventas','pottot','cyg','va','c_v','cod_dpto','cpam','grupos12','anos_espacio','hombre'] 

'''
Creación de variables
'''

# Crear la variable de Costos y Gastos agregada 
df['cyg'] = df[['gasto_tot','costos']].sum(axis=1)
df.loc[(df['gasto_tot'].isna()) & (df['costos'].isna()), 'cyg'] = np.nan

# Total Personal Ocupado
df['trabajsoc'] = pd.to_numeric(df['trabajsoc'], errors= 'coerce')
df['trabajpago'] = pd.to_numeric(df['trabajpago'], errors= 'coerce')
df['trabajfami'] = pd.to_numeric(df['trabajfami'], errors= 'coerce')

df['pottot'] = df[['trabajsoc','trabajpago','trabajfami']].sum(axis=1)
df.loc[(df['trabajsoc'].isna()) & (df['trabajpago'].isna()) & (df['trabajfami'].isna()), 'cyg'] = np.nan

# Ciudad Principal y Área Metropolitana
cpam = [11001,	5001,	5079,	5088,	5129,	5212,	5266,	5308,	5360,	5380,	5631,	76001,	76892,	8001,
        8758,	68001,	68276,	68307,	68547,	17001,	17873,	52001,	66001,	66170,	66400,	73001,	54001,	54874,
        54405,	54261,	50001,	23001,	13001,	15001,	18001,	19001,	20001,	27001,	41001,	44001,	47001,	63001,	
        70001,	88001]
df['cod_mpio'] = pd.to_numeric(df['cod_mpio'], errors= 'coerce')
df['cpam'] = (df['cod_mpio'].isin(cpam) & df['clase']==1).apply(lambda x: int(x))

# Actividad Económica
df['division'] = df['ciiu_4'].apply(Funciones.extract_first_two_digits)
df = pd.merge(df,rel_g12,on='division',how="left")

# Valor Agregado
df['va'] = df['ventas'] - df['cyg']

# Coeficiente técnico de costos y gastos sobre ingresos
df['c_v'] = df['cyg'] / (df['ventas'] + 1)

'''
Modificación de los labels de algunas variables
'''
# Antigüedad
df['anos_espacio'] = df['anos_espacio'].map(dict(zip(rel_anio['ANOS_ESPACIO'], rel_anio['Codigo'])))

# Sexo
df['sexo'] = df['sexo'].map(dict(zip(rel_sex['SEXO'], rel_sex['Codigo'])))
df.rename(columns = {'sexo' : 'hombre'}, inplace = True)


'''
Selección de las variables
'''
df_final = df[var_mov]

In [36]:
df_final.shape

(3475, 11)

0        2. De 1 a menos de 3 años
1        3. De 3 a menos de 5 años
2                1. Menos de 1 año
3                 5. 10 años y más
4        2. De 1 a menos de 3 años
                   ...            
3470              5. 10 años y más
3471     3. De 3 a menos de 5 años
3472             1. Menos de 1 año
3473              5. 10 años y más
3474    4. De 5 a menos de 10 años
Name: anos_espacio, Length: 3475, dtype: object